In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
from dotenv import load_dotenv
load_dotenv(override=True)

True

In [10]:
import os
from openai import AsyncAzureOpenAI
from semantic_kernel.kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

# See https://techcommunity.microsoft.com/blog/azuredevcommunityblog/using-keyless-authentication-with-azure-openai/4111521
credential = DefaultAzureCredential()
token_provider = get_bearer_token_provider(
    credential, "https://cognitiveservices.azure.com/.default"
)


def create_kernel(service_id: str = "default") -> Kernel:
    kernel = Kernel()

    client = AsyncAzureOpenAI(
        azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
        azure_deployment=os.getenv("AZURE_OPENAI_MODEL"),
        azure_ad_token_provider=token_provider,
        api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
    )

    kernel.add_service(
        AzureChatCompletion(
            deployment_name=os.getenv("AZURE_OPENAI_MODEL"),
            async_client=client,
            service_id=service_id,
        )
    )

    return kernel


In [11]:
from typing import Annotated
from semantic_kernel.agents import ChatCompletionAgent
from semantic_kernel.connectors.ai import (
    FunctionChoiceBehavior,
    PromptExecutionSettings,
)
from semantic_kernel.functions import kernel_function, KernelArguments
import json


class BillingAgentPlugin:
    @kernel_function(description="Retrieve payment methods for a customer")
    def get_payment_methods(
        self,
        customer_id: Annotated[
            str, "The ID of the customer to get the payment methods for"
        ],
    ) -> Annotated[str, "The payment methods available for the specified customer"]:
        return "credit_card, paypal, bank_transfer"


billing_agent_kernel = create_kernel()

billing_agent_kernel.add_plugin(BillingAgentPlugin(), plugin_name="BillingAgent")
settings = billing_agent_kernel.get_prompt_execution_settings_from_service_id(
    service_id='default')
settings.function_choice_behavior = FunctionChoiceBehavior.Auto()

billing_agent = ChatCompletionAgent(
    description="A billing support agent that can answer billing-related questions, like invoices, payment methods, and usage metrics",
    id="billing",
    name="Billing",
    kernel=billing_agent_kernel,
    arguments=KernelArguments(settings=settings),
    instructions="""
    You are a billing support agent that responds to customer inquiries.
    
    Your tasks are:
    - Retrieve the last invoice for a customer
    - Provide available payment methods for a customer
    - Change the payment method for a customer
    - Provide usage metrics for telco services (voice, data, SMS)
    - Handle disputes and billing inquiries
    - Provide billing-related information and support
    
    Rules:
    - Make sure to act politely and professionally.
    """,
)


In [12]:
# # Single agent works
# from semantic_kernel.contents import ChatHistory

# history = ChatHistory()
# history.add_user_message("My customer code is 12345. Tell me my usage metrics")

# async for result in billing_agent.invoke(history=history):
#     print(f"Result: {result}")

In [13]:
from semantic_kernel.agents import AgentGroupChat
from semantic_kernel.agents.strategies import SequentialSelectionStrategy, DefaultTerminationStrategy

chat = AgentGroupChat(
        agents=[billing_agent],
        termination_strategy=DefaultTerminationStrategy(maximum_iterations=2),
        selection_strategy=SequentialSelectionStrategy(),
    )


In [14]:
from semantic_kernel.contents import AuthorRole, ChatMessageContent
await chat.reset()
await chat.add_chat_message(ChatMessageContent(role=AuthorRole.USER, content="My customer code is 12345. Tell me my payment method"))
async for result in chat.invoke():
    print(f"Result: {result}")

Result: Your available payment methods are:
- Credit Card
- PayPal
- Bank Transfer

Let me know if you'd like to make any changes to your payment method!
Result: Your available payment methods are:
- Credit Card
- PayPal
- Bank Transfer

If you'd like to know your current payment method, let me know!
